<a href="https://colab.research.google.com/github/my000own000files1/Fuzzy/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-fuzzy

In [ ]:
!pip install -U control

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import control as ct
from control.matlab import * #I want to have a MATLAB-like environment
import matplotlib.pyplot as plt

In [ ]:
#creating object dynamics
def port_crane_updfcn(t,x,u,params={})
  """A port crane with a cargo on a rope dynamics (like object) 
  assumptions: by all perturbations such as the rolling friction
   force at the point of contact with the rails, air resistance,
    we also assume that the cable is inextensible, i.e. L=const

  Parameters
  ----------
  x: array 
      System state: [x,dx,phi,dphi], where x is a port crane (object) position
      in [0,200] [m], dx is velocity of the port crane in m/s, phi is an angle
      of deflection of the rope with a container, which is a cargo and attached
      to the end of the rope in rad, dphi is angular velocity of the rope 
      deflection in rad/sec

  u: array
      System inputs: [Fx], where Fx is a pulling force exerted on the
      trolley, created by electric motor in Newton

  Returns 
  -------
  float
      [ddx,ddphi], where ddx is a port crane acceleration in m/s^2 and ddphi is 
      angular acceleration in rad/s^2
  """

  from math import son,cos

  # Set up the system parameters
  m = params.get('m', 4000.)  # cargo mass in kg
  M = params.get('M', 10000.) # port crane mass in kg
  g = params.get('g', 9.8)    # gravitational constant, m/s^2
  L = params.get('L', 30)     # length of the drope in m

  # Define variables for object state
  x = x[0]    # port crane position in m
  dx = x[1]   # velocity of the port crane in m/s
  phi = x[2]  # deflection of the rope angle in rad
  dphi = x[3] # angular velocity of the rope deflection in rad/sec

  #Define variables for object inputs
  Fx = u[0]   # pulling force exerted on the trolley in Newtons

  # calculating the output variables
  ddx = (Fx + m*sin(phi)*(L*dphi**2+g*cos(phi))/(M + m*(sin(phi))**2)
  ddphi = ((-1)/L)*(ddx*cos(phi) + g*sin(phi))

  return [dx,ddx,dphi,ddphi]

#Input/ouput model for the object
ct.NonlinearSystem((port_crane_updfcn, None), name='object',
                   inputs = ('Fx'), outputs = ('x,phi'), states=('x,dx,phi,dphi'))

#define the generator for simulating an aoutonomus object, when state var
# equal to null. And also Fx = constant
def system_dynamic_plot(sys, t, y, t_hill=5, xref=100, linetype='b-',
                         subplots=[None, None]):
  #Figure out the plot bounds and indices
  x_min = 0; x_max = x_ref+10; x_ind = sys.find_ouput('x')
  phi_min = -math.pi/4; phi_max = math.pi/4; phi_ind = sys.find_output('phi')

  # Make sure the upper and lower bounds on x are OK
  while max(y[x_ind]) > x_max: x_max += 1

  # Make sure the upper and lower bounds on phi are OK
  while max(y[phi_ind]) > phi_max: phi_max += 1
  while min(y[phi_ind]) < phi_min: phi_min -= 1

  # Position profile
  if subplot_axes[0] is None
    subplot_axes[0] = plt.subplot(2,1,1)
  else
    plt.sca(subplots[0])
  plt.plot(t, y[x_ind], linetype)
  plt.plot(t, xref*np.ones(t.shape), 'k-')
  plt.plot([t_hill, t_hill], [x_min, x_max], 'k--')
  plt.axis([0, t[-1], x_min, x_max])
  plt.xlabel('Time $t$ [s]')
  plt.ylabel('Port crane position $x$')
  
  # Angle phi profile
  if subplot_axes[1] is None
    subplot_axes[1] = plt.subplot(2,1,2)
  else
    plt.sca(subplots[1])
  plt.plot(t, t[phi_ind], linetype)
  plt.plot([t_hill, t_hill], [phi_min, phi_max], 'k--')
  plt.axis([0, t[-1], phi_min, phi_max])
  plt.xlabel('Time $t$ [s]')
  plt.ylabel('deflection of the rope angle $phi$')
  
  # Applied input profile
  # if an   
  return subplot_axes

In [ ]:
def port_crane_outdcn(t ,x ,u , param={})
  # Get the port crane parameters that we need
  